In [1]:
import base64
import requests
import json
import pandas as pd
import flat_table
from config import consumer_key,consumer_secret

In [2]:
consumer_key_secret = consumer_key+":"+consumer_secret
consumer_key_secret_enc = base64.b64encode(consumer_key_secret.encode()).decode()

In [3]:
headersAuth = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Authorization': 'Basic '+ str(consumer_key_secret_enc)
    }

data = {
    'grant_type': 'client_credentials',
    'scope' : 'product.compact'
    }

## Authentication request
response = requests.post('https://api.kroger.com/v1/connect/oauth2/token', headers=headersAuth, data=data, verify=True)
j = response.json()

access_token = (j['access_token'])


In [9]:
def get_milk(): 
    
    # initialization
    page = 1
    start=1
    output = []
        
    while page<21: 

        # run the request
        headersAPI = {
            'accept': 'application/json',
            'Authorization': 'Bearer '+access_token
            }

        # run the request
        http = "https://api.kroger.com/v1/products?filter.term={{"
        term = '"milk"'
        http2 = "}}&filter.limit="
        limit=50
        http3 = "&filter.start="
        start = 1
        
        url = http+term+http2+str(limit)+http3+str(start)

        response = requests.get(url,headers=headersAPI, verify=True).json()           

                    
        # add response data to output
        
        output += response['data']
        
        # handle pagination
        page += 1
        start+=50
    return output    

In [10]:
milk_list = get_milk()
with open('milk_list.json', 'w') as f:
    json.dump(milk_list, f)

In [11]:
milk_list

[{'productId': '0001111040101',
  'upc': '0001111040101',
  'aisleLocations': [],
  'brand': 'Kroger',
  'categories': ['Dairy'],
  'description': 'Kroger® Vitamin D Whole Milk',
  'images': [{'perspective': 'left',
    'sizes': [{'size': 'small',
      'url': 'https://www.kroger.com/product/images/small/left/0001111040101'},
     {'size': 'large',
      'url': 'https://www.kroger.com/product/images/large/left/0001111040101'},
     {'size': 'xlarge',
      'url': 'https://www.kroger.com/product/images/xlarge/left/0001111040101'},
     {'size': 'thumbnail',
      'url': 'https://www.kroger.com/product/images/thumbnail/left/0001111040101'},
     {'size': 'medium',
      'url': 'https://www.kroger.com/product/images/medium/left/0001111040101'}]},
   {'perspective': 'front',
    'featured': True,
    'sizes': [{'size': 'xlarge',
      'url': 'https://www.kroger.com/product/images/xlarge/front/0001111040101'},
     {'size': 'small',
      'url': 'https://www.kroger.com/product/images/small/

In [12]:
#Normalize the json data into a data frame and expand the items category to pick up the container size
df = pd.json_normalize(milk_list,"items",["productId", "categories","description"])

df.head(3)

,itemId,favorite,size,fulfillment.curbside,fulfillment.delivery,fulfillment.inStore,fulfillment.shipToHome,productId,categories,description
0,0001111040101,False,1 gal,False,False,False,False,0001111040101,[Dairy],Kroger® Vitamin D Whole Milk
1,0001111041600,False,1/2 gal,False,False,False,False,0001111041600,[Dairy],Kroger® 2% Reduced Fat Milk
2,0001111041660,False,1 gal,False,False,False,False,0001111041660,[Dairy],Kroger® 1% Lowfat Milk


In [13]:
#Select only the columns we're wanting to keep
df = df[['productId','categories','description','size']]

df.head(3)

,productId,categories,description,size
0,0001111040101,[Dairy],Kroger® Vitamin D Whole Milk,1 gal
1,0001111041600,[Dairy],Kroger® 2% Reduced Fat Milk,1/2 gal
2,0001111041660,[Dairy],Kroger® 1% Lowfat Milk,1 gal
